# Data Access & Exploration

## 0. Imports

In [6]:
import os.path
from os.path import isfile, join
import os
from os import listdir
from osgeo import gdal
import errno
import numpy as np
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.windows import from_bounds #Window
from rasterio.enums import Resampling
from sys import platform

In [7]:
# Load config
from config import UNetTraining
# from config import UNetTraining
config = UNetTraining.Configuration()

In [8]:
from config import colors
colors = colors.bcolors()

In [9]:
if platform == "linux" or platform == "linux2":
    path_satellite = "/home/jovyan/work/satellite_data/"
    path_labels = os.getcwd() + "/1_labeled_data/tif/"
    cutouts_path = os.getcwd() + "/2_cutouts/"
elif platform == "darwin":
    path_data = "/Users/leori/Desktop/BA/1_Data/"
    path_satellite = path_data + "2_satellite/"
    path_labels = path_data + "1_labeled_data/tif/"
    cutouts_path = None
elif platform == "win32":
    print("Something went wrong")

# 1. Data Inspection

### 1.1 Functions

In [10]:
def inspect_tif_resolution(filepath):
    img = rasterio.open(filepath)
    xRes, yRes = img.res
    print("Resolution: {} x {}".format(xRes, yRes))

In [11]:
def bounds(filepath):
    src_img = rasterio.open(filepath)
    print(src_img.bounds)
    left, bottom, right, top = src_img.bounds[0],src_img.bounds[1], src_img.bounds[2], src_img.bounds[3]


In [39]:
def inspect_tif(dataset):
    print("Metadata:", dataset.GetMetadata())
    print("Projection: ", dataset.GetProjection())
    print("Size is {} x {} x {}".format(dataset.RasterXSize,
                                        dataset.RasterYSize,
                                        dataset.RasterCount))
    geotransform = dataset.GetGeoTransform()
    if geotransform:
        print("Origin = ({}, {})".format(geotransform[0], geotransform[3]))
        print("Pixel Size = ({}, {})".format(geotransform[1], geotransform[5]))

    # Raster Band
    band = dataset.GetRasterBand(1)
    min = band.GetMinimum()
    max = band.GetMaximum()
    if not min or not max:
        (min,max) = band.ComputeRasterMinMax(True)
    print("Min={:.3f}, Max={:.3f}".format(int(min),int(max)))
        
    size_x_px = dataset.RasterXSize
    size_y_px = dataset.RasterYSize
    pixel_size = (dataset.GetGeoTransform()[1], dataset.GetGeoTransform()[5])
    size_x = size_x_px * pixel_size[0]
    size_y = size_y_px * pixel_size[1]
    print("# Pixels in Image = ({}, {}) with a Raster Count of {}".format(size_x_px, size_y_px, dataset.RasterCount))
    mult_to_m = 100000
    mult_to_km = mult_to_m / 1000
    print("Size of one Pixel in m = {}".format(tuple([mult_to_m*x for x in pixel_size])))
    print("Size Covered by Image / Image Size = ({}km, {}km)".format(round(mult_to_km*size_x, 2), abs(round(mult_to_km*size_y, 2))))

In [13]:
def isfile(filepath: str = None):
    # if filepath does not exists, an error is thrown
    if not os.path.isfile(filepath): raise FileNotFoundError(errno.ENOENT, os.strerror(errno.ENOENT), filepath) #print("File does not exist.")

In [14]:
def check_filesize(filepath, size_in_mb: int = 500):
    filesize_in_mb = os.path.getsize(filepath) / 1000000
    if filesize_in_mb > size_in_mb: 
        raise Exception("File is too large: {}MB".format(int(filesize_in_mb)))

In [15]:
def preview_tif(tif_filepath: str = None):
    isfile(tif_filepath)
    check_filesize(tif_filepath, 500)
    img = rasterio.open(tif_filepath)
    if img: show(img)
    else: print("img unable to be loaded at this moment")

### 1.2 Data Inspection

ToDos Data Preparation
- Data Cleaning (visualize data first): rm redundant information & set no data values to something - some metadata?
- Custom Transformers
- Feature Scaling


⇒ Pipeline (cf. first example in `4_pipeline.ipynb`)

In [16]:
filepath_label = config.filepath_label
filepath_satellite = config.filepath_satellite

In [ ]:
satel 29.746877767434313, -26.247113163017453
label 29.746877767,       -26.247113163

In [38]:
print("Satellite")
inspect_tif(gdal.Open(filepath_satellite))
bounds(filepath_satellite)

print("\nLabel")
inspect_tif(gdal.Open(filepath_label))
bounds(filepath_label)

Satellite
Metadata: {'ALL_COMMENTS': 'Created by OpenJPEG version 2.4.0', 'AREA_OR_POINT': 'Area', 'COLORSPACE': 'RGB', 'COMPRESSION_RATE_TARGET': '1', 'TIFFTAG_MAXSAMPLEVALUE': '0', 'TIFFTAG_MINSAMPLEVALUE': '0', 'TIFFTAG_RESOLUTIONUNIT': '1 (unitless)', 'TIFFTAG_XRESOLUTION': '0', 'TIFFTAG_YRESOLUTION': '0'}
Projection:  GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
Size is 108224 x 107798 x 3
Origin = (29.746877767434313, -26.247113163017453)
Pixel Size = (2.3728143101288307e-06, -2.3728143101288515e-06)
Min=0.000, Max=255.000
# Pixels in Image = (108224, 107798) with a Raster Count of 3
Size of one Pixel in m = (0.23728143101288307, -0.23728143101288515)
Size Covered by Image / Image Size = (25.68km, 25.58km)
BoundingBox(left=29.746877767434313, bottom=-

In [ ]:
2.3728143101288307e-06, -2.3728143101288515e-06
2.372814311058536e-06,  -2.372814310098498e-06

- Size of one Pixel in cm = (0.23728143101288307, -0.23728143101288515)
- Size of one Pixel in cm = (0.23728143099711865, -0.23728143123798187)

- Size Covered by Image / Image Size = (256.8m, 255.78m)
- Size Covered by Image / Image Size = (252.93m, 251.7m)

- BoundingBox(left=29.746877767434313, bottom=-26.502897800020722, right=30.003673223333696, top=-26.247113163017453)
- BoundingBox(left=29.748224553,       bottom=-26.50038228,        right=30.00115944,        top=-26.248683629)

- Size is 108224 x 107798 x 3
- Size is 106597 x 106076 x 1

In [18]:
preview_tif(filepath_label)
preview_tif(filepath_satellite)

Exception: File is too large: 46667MB

In [ ]:
cutout_window = (29.748021463931796, -26.24839922837354, 29.74923634485858, -26.249614109300325)

In [ ]:
check_filesize(filepath_label, 1000)
src_label = rasterio.open(filepath_label)
print("shape:", src_label.read().shape)
# src_label.colorinterp[3] # rgba

Exception: File is too large: 45232MB

In [ ]:
check_filesize(filepath_satellite, 1000)
src_satellite.read().shape
# src_satellite.read(2)[1884-1][3050-1] # blue pixel value in bottom left corner

Exception: File is too large: 2062MB

In [ ]:
from PIL import Image
import numpy as np

img = Image.open(filepath_label)

# convert to numpy array
img = np.array(img)

# find number of channels
if img.ndim == 2:
    channels = 1
    print("image has 1 channel")
else:
    channels = img.shape[-1]
    print("image has", channels, "channels")

image has 4 channels


### 1.3 Data Visualization

In [ ]:
inspect_tif_resolution(filepath_satellite)
inspect_tif_resolution(filepath_label)

Resolution: 2.3728143101288307e-06 x 2.3728143101288515e-06
Resolution: 2.3728143099711865e-06 x 2.3728143123798187e-06


In [ ]:
inspect_tif(gdal.Open(filepath_label))

Size is 30254 x 30188 x 4
Origin = (29.748224553, -26.248679767728067)
Pixel Size = (8.335734183909613e-06, -8.335734183909613e-06)
Min=0.000, Max=255.000
# Pixels in Image = (30254, 30188) with a Raster Count of 4
Size of one Pixel in cm = (0.8335734183909613, -0.8335734183909613)
Size Covered by Image / Image Size = (252.19m, 251.64m)
